In [1]:
## Packages need for data pre-process
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

from scipy import sparse
from collections import Counter

import nltk
from numpy import savetxt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import itertools

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# For evaluztion
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import cohen_kappa_score

# Running time
from timeit import default_timer as timer

# Visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure

In [20]:
doc_metadata = pd.read_csv("/home/rep/scRNA-seq_clustering_to_Twitter/P3_Proposed_workflow/PW_files/doc_metadata_stemming_valijobs_hclabeled.csv")
del doc_metadata['Unnamed: 0']

In [21]:
doc_metadata

,user,tweet,HCCA_category,true_label,HCCA_label,true_category
0,Accounting jobs,"#jobs4u #jobs ACCOUNTING CLERK, [Baton Rouge, ...",Advertisement,2,2,Advertisement
1,TMJ-SAT Retail Jobs,This tweet has been removed in accordance with...,Advertisement,2,2,Advertisement
2,Dallas jobs,#jobs4u #jobs Construction Project Manager - R...,Advertisement,2,2,Advertisement
3,Massachusetts Jobs,"Recruiter needed in #Braintree, #MA https://t....",Advertisement,2,2,Advertisement
4,☆Assata☆,@CTheLily Lol oh girl I aint goin down there a...,Personal,5,5,Personal
...,...,...,...,...,...,...
791,haruszpapa,current.tv: supernews...gates vs jobs...awesome!,Junk,1,1,Junk
792,TravelManager1,Tourism and #Economy Saudi- Tourism sector to ...,Other,3,4,News/Politics
793,magneda2,"Reuters: GE Capital to cut up to 11,000 jobs: ...",Other,3,4,News/Politics
794,price_laborecon,RT @escapetochengdu: Women are 50% more likely...,Personal,4,5,Other


#### - With the "Other" group

In [22]:
True_Label = list(doc_metadata["true_label"])
# (1) Purity
Correct_target = pd.DataFrame(doc_metadata[["true_category", "true_label"]])
numerator = doc_metadata.groupby(["HCCA_label", "true_label"], as_index=False)['true_category'].count().sort_values('true_category', ascending=False).drop_duplicates("HCCA_label")["true_category"].sum()
purity = numerator/len(True_Label)
print("purity (with `Other`):", purity)

# (2) AMI
AMI = adjusted_mutual_info_score(True_Label, doc_metadata["HCCA_label"])
print("AMI (with `Other`):", AMI)

# (3) ARI
ARI = adjusted_rand_score(True_Label, doc_metadata["HCCA_label"])
print("ARI (with `Other`):", ARI)

# (4) Cohen's kappa
cohen_kappa = cohen_kappa_score(True_Label, doc_metadata["HCCA_label"])
print("Cohen's kappa (with `Other`):", cohen_kappa)

purity (with `Other`): 0.7399497487437185
AMI (with `Other`): 0.4302011268215467
ARI (with `Other`): 0.48456769866443944
Cohen's kappa (with `Other`): 0.5883149011731647


#### - Without the "Other" group

In [57]:
# re-organize HCCA_label
doc_metadata_new = doc_metadata[doc_metadata.true_category != "Other"]
correspondence = doc_metadata_new.groupby(['HCCA_label', 'true_label'], as_index=False)['true_category'].count().sort_values('true_category', ascending=False).drop_duplicates('HCCA_label')
HCCA_label = correspondence.HCCA_label.to_list()
true_label = correspondence.true_label.to_list()
d = dict(zip(HCCA_label, true_label))
doc_metadata_new["HCCA_label"].replace(d, inplace=True)

In [59]:
True_Label = list(doc_metadata_new["true_label"])
# (1) Purity
Correct_target = pd.DataFrame(doc_metadata_new[["true_category", "true_label"]])
numerator = doc_metadata_new.groupby(["HCCA_label", "true_label"], as_index=False)['true_category'].count().sort_values('true_category', ascending=False).drop_duplicates("HCCA_label")["true_category"].sum()
purity = numerator/len(True_Label)
print("purity (with `Other`):", purity)

# (2) AMI
AMI = adjusted_mutual_info_score(True_Label, doc_metadata_new["HCCA_label"])
print("AMI (with `Other`):", AMI)

# (3) ARI
ARI = adjusted_rand_score(True_Label, doc_metadata_new["HCCA_label"])
print("ARI (with `Other`):", ARI)

# (4) Cohen's kappa
cohen_kappa = cohen_kappa_score(True_Label, doc_metadata_new["HCCA_label"])
print("Cohen's kappa (with `Other`):", cohen_kappa)

purity (with `Other`): 0.7906040268456376
AMI (with `Other`): 0.4620807647726442
ARI (with `Other`): 0.5746455629998027
Cohen's kappa (with `Other`): 0.6696211815406308
